In [96]:
%matplotlib inline

In [208]:
import pennylane as qml
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon, Patch
from itertools import product
from dataclasses import dataclass

import numpy as np
np.set_printoptions(suppress=True)

@dataclass(frozen=True)
class Wire:
    layer: int
    i: int
    j: int

## Surface code

### Raw lattice definition

In [222]:
## lattice dimensions

## Qubit array layers
n_qubit_layers = 3 # | 0: Surface code | 1: type-I auxiliar qubits | 2: type-II auxiliar qubits |

## surface code sites
nx = 2
ny = 2
surface_code_all_sites = [(0, px, py) for px, py in product(range(nx), range(ny))]

## type-I auxiliar qubits sites
nx_I = 0 # nx - 1
ny_I = 0 # ny - 1
type_I_all_sites = [(1, px, py) for px, py in product(range(nx_I), range(ny_I))]

## type-II auxiliar qubits sites
nx_II = 0 # nx_I // 2
ny_II = 0 # ny_I // 2
type_II_all_sites = [(2, px, py) for px, py in product(range(nx_II), range(ny_II))]

## raw qubit lattice 
#dev = qml.device("lightning.qubit", wires=[Wire(*s) for s in surface_code_all_sites]) # debug
dev = qml.device("lightning.qubit", wires=[Wire(*s) for s in surface_code_all_sites + type_I_all_sites + type_II_all_sites])

### Data qubits

In [223]:
## Data qubits
data_sites = []

for x, y in all_sites:
    x0 = 2*x 
    y0 = 2*y 
    
    data_a_site = (0, x0, y0)
    data_b_site = (0, x0 + 1, y0 +1)
    
    if (data_a_site[1] < nx) and (data_a_site[2] < ny):
        data_sites.append(data_a_site)
        
    if (data_b_site[1] < nx) and (data_b_site[2] < ny):
        data_sites.append(data_b_site)

### X-measurement qubits

In [224]:
## X measurement sites and operators
x_measure_sites = []
x_measure_data_sites = [] 
x_op = []

for x, y in product(range(nx // 2 + nx % 2), range(ny // 2 + ny % 2)):
    x0 = 2 * x
    y0 = 2 * y + 1
    
    sites = []
    for px, py in [(x0 - 1, y0), (x0, y0 - 1), (x0 + 1, y0), (x0, y0 + 1)]:
        if ((px >= 0) and (py >= 0)) and ((px < nx) and (py < ny)):
            sites.append((0, px, py))
            
    op = qml.operation.Tensor(*(qml.PauliX(Wire(*s)) for s in sites))

    x_measure_sites.append((0, x0, y0))
    x_measure_data_sites.append(sites)
    x_op.append(op)

### Z-measurement qubits

In [225]:
## Z measurement sites and operators
z_measure_sites = []
z_measure_data_sites = [] 
z_op = []

for x, y in product(range(nx // 2 + nx % 2), range(ny // 2 + ny % 2)):
    x0 = 2 * x + 1
    y0 = 2 * y 
    
    sites = []
    for px, py in [(x0 - 1, y0), (x0, y0 - 1), (x0 + 1, y0), (x0, y0 + 1)]:
        if ((px >= 0) and (py >= 0)) and ((px < nx) and (py < ny)):
            sites.append((0, px, py))
            
    op = qml.operation.Tensor(*(qml.PauliZ(Wire(*s)) for s in sites))

    z_measure_sites.append((0, x0, y0))
    z_measure_data_sites.append(sites)
    z_op.append(op)

In [226]:
x_measure_sites

[(0, 0, 1)]

In [227]:
z_measure_sites

[(0, 1, 0)]

In [228]:
data_sites

[(0, 0, 0), (0, 1, 1)]

In [229]:
x_op

[PauliX(wires=[Wire(layer=0, i=0, j=0)]) @ PauliX(wires=[Wire(layer=0, i=1, j=1)])]

### Code cycle
#### Random initial state for the data qubits

In [230]:
from scipy.stats import rv_continuous

class sin_prob_dist(rv_continuous):
    def _pdf(self, theta):
        return 0.5 * np.sin(theta)

# Samples of theta should be drawn from between 0 and pi
sin_sampler = sin_prob_dist(a=0, b=np.pi)

@qml.qnode(dev)
def haar_random_unitary(layer, sx, sy):
    phi, omega = 2 * np.pi * np.random.uniform(size=2) # Sample phi and omega as normal
    theta = sin_sampler.rvs(size=1) # Sample theta from our new distribution
    qml.Rot(phi, theta, omega, wires=Wire(layer, sx, sy))
    return qml.state()

for site in data_sites:
    haar_random_unitary(*site)
    
@qml.qnode(dev)
def state():
    return qml.state()

#### Surface code implementation

In [231]:
@qml.qnode(dev)
def surface_code():
    ## measurement qubits init
    ### x-syndrome initial Hadamard and x-controlled CNOT      
    for n_site, x_m_site in enumerate(x_measure_sites):
        qml.Hadamard(Wire(*x_m_site))
        op = x_op[n_site]
        for target in op.wires:
            qml.CNOT(wires=[Wire(*x_m_site), target])
        
    ### z-target CNOT
    for n_site, z_m_site in enumerate(z_measure_sites):
        op = z_op[n_site]
        for target in op.wires:
            qml.CNOT(wires=[target, Wire(*z_m_site)])
            
    ### x-syndrome final Hadamard  
    for x_m_site in x_measure_sites:
        qml.Hadamard(Wire(*x_m_site))
    
    return qml.state()

In [232]:
print(qml.draw(surface_code)())

Wire(layer=0, i=0, j=0): ────╭X────╭●────┤  State
Wire(layer=0, i=0, j=1): ──H─╰●─╭●─│───H─┤  State
Wire(layer=0, i=1, j=0): ───────│──╰X─╭X─┤  State
Wire(layer=0, i=1, j=1): ───────╰X────╰●─┤  State


In [233]:
surface_code()

tensor([ 0.5+0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0.5+0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0.5+0.j,  0. +0.j,  0. +0.j,
         0. +0.j, -0.5+0.j,  0. +0.j,  0. +0.j], requires_grad=True)

### Check 1

In [179]:
base = [np.array([1,0]), np.array([0,1])]

def state_vec(i,j,k,l):
    return np.kron(base[i],np.kron(base[j], np.kron(base[k], base[l])))

.5*(state_vec(0,0,0,0) + state_vec(1,0,0,0) + state_vec(0,1,1,0) - state_vec(1,1,1,0))

@qml.qnode(dev_test)
def code_test():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[0,2])
    qml.CNOT(wires=[1,3])
    qml.CNOT(wires=[2,3])
    qml.Hadamard(wires=0)
    return qml.state()

array([ 0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  0.5,  0. ,  0. ,
        0. ,  0. ,  0. , -0.5,  0. ])

In [ ]:
print(qml.draw(code_test)())

In [193]:
code_test()

tensor([ 0.5+0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
         0.5+0.j,  0. +0.j,  0.5+0.j,  0. +0.j,  0. +0.j,  0. +0.j,
         0. +0.j,  0. +0.j, -0.5+0.j,  0. +0.j], requires_grad=True)

In [180]:
## raw qubit lattice 
dev_test = qml.device("lightning.qubit", wires=4)

## Holonomic gate implementation

### Single qubit gate

In [ ]:
# Holomonic gates based on https://journals.aps.org/prapplied/abstract/10.1103/PhysRevApplied.13.014055

## Single qubit case J_A2 = 0


## Two-qubit case J_A2 \neq 0

In [ ]:
coeffs = [1, 1]
obs = [qml.PauliX(0), qml.PauliX(1)]
hamiltonian = qml.Hamiltonian(coeffs, obs)

@qml.qnode(dev)
def circuit(time):
    ApproxTimeEvolution(hamiltonian, time, 1)
    return [qml.expval(qml.PauliZ(wires=i)) for i in wires]